In [67]:
from glob import glob
import rasterio
import numpy as np
import itertools
import math

# Calculating Variance of each band

In [68]:
def get_variance(bands):
    variance_bands = []
    for img in bands:
        with rasterio.open(img, 'r') as f:
            arr = np.array(f.read(1))
            N = arr.shape[0] * arr.shape[1]
            a = np.multiply(arr, arr)
            a = np.sum(a)
            a = a / N

            b = np.sum(arr) * np.sum(arr)
            b = b / (N * N)
            variance_bands.append(a - b)
    return variance_bands

# Find all possible subsets of given size

In [69]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

# Calculate Covariance matrix

In [70]:
def get_covariance_matrix(bands_combination):
    covariance_bands = []
    for b in bands_combination:
        b1, b2 = None, None
        with rasterio.open(bands[b[0]], 'r') as f:
            b1 = np.array(f.read(1))
        
        with rasterio.open(bands[b[1]], 'r') as f:
            b2 = np.array(f.read(1))
        
        N = b1.shape[0] * b1.shape[1]

        a = np.multiply(b1, b2)
        a = np.sum(a)
        a = a / N

        b = np.sum(b1) / N
        c = np.sum(b2) / N

        covariance_bands.append(a - (b * c))
    return covariance_bands

# Calculate Correlation Matrix

In [71]:
def get_correlation_matrix(bands_combination, covariance_bands):
    correlation_bands = np.zeros((len(bands_combi_2), len(bands_combi_2)))
    i = 0
    for b in bands_combi_2:
        b1, b2 = b
        correlation_bands[b1][b2] = covariance_bands[i] / math.sqrt(variance_bands[b1] * variance_bands[b2])
        i = i + 1
    return correlation_bands

# Calculate Standard Deviation

In [72]:
def get_std_bands(variance_bands):
    return np.sqrt(variance_bands)

In [73]:
# def calculate_oif(combs_3, correlation_bands, std_bands, band_names):
#     oif_history = []
#     for b in combs_3:
#         b1, b2, b3 = b
#         x, y, z = correlation_bands[b1][b2], correlation_bands[b1][b3], correlation_bands[b2][b3], 
#         oif = (std_bands[b1] + std_bands[b2] + std_bands[b3]) / (np.abs(x) + np.abs(y) + np.abs(z))
#         oif_history.append(f'OIF [Band Models = ({band_names[b1]}, {band_names[b2]}, {band_names[b3]})] = {oif}')
#         #print(f'OIF [Band Models = ({band_names[b1]}, {band_names[b2]}, {band_names[b3]})] = {oif}')
#     return oif_history

In [85]:
def calculate_oif(combs, correlation_bands, std_bands, band_names):
    oif_history = []
    for b in combs:
        N = len(b)

        std_sum = 0
        for i in range(N):
            std_sum += std_bands[b[i]]
        corre = 0
        for i in range(N - 1):
            for j in range(i + 1, N):
                corre += np.abs(correlation_bands[b[i]][b[j]])
        
        oif = std_sum / corre
        band_str = ''

        for i in range(N - 1):
            band_str += band_names[b[i]]
            band_str += ', '
        band_str += band_names[b[N - 1]]

        oif_history.append(f'[Band Models = {band_str}], OIF = {oif}')
    return oif_history

# Selecting bands and Getting Data

In [86]:
area = 'Area_one/Tiles/'
tile_number = 'T1/'
meters = ['D10m', 'D20m']

In [87]:
def build_dependencies(area, tile_number, meters):
    # Getting Bands
    bands = []
    for m in meters:
        imagePath = area + tile_number + m + '/'
        bs = glob(imagePath + '*B*.tif')
        bands += bs
    bands.sort()

    # Setting band names and indices
    band_names = [b[-11:-5] for b in bands]
    band_numbers = [i for i in range(len(band_names))]

    # Calculate Variance
    variance_bands = get_variance(bands)

    # Bands Combination (Size = 2)
    combs_2 = findsubsets(band_numbers, 2)

    # Calculate covariance of combinations
    covariance_bands = get_covariance_matrix(combs_2)

    # Build Correlation Matrix
    correlation_bands = get_correlation_matrix(combs_2, covariance_bands)

    # Calculate Standard Deviation
    std_bands = get_std_bands(variance_bands)

    return correlation_bands, std_bands, band_names, band_numbers

In [88]:
correlation_bands, std_bands, band_names, band_numbers = build_dependencies(area, tile_number, meters)

/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in ulong_scalars
  # This is added back by InteractiveShellApp.init_path()


# 3 bands OIF Calculation

In [94]:
combs_3 = findsubsets(band_numbers, 3)
oif_combs_3 = calculate_oif(combs_3, correlation_bands, std_bands, band_names)

In [103]:
oif_combs_3.sort()
print(oif_combs_3[0])
print(oif_combs_3[1])

[Band Models = B02_10, B02_20, B03_20], OIF = 7.596282858914765
[Band Models = B02_10, B02_20, B04_20], OIF = 6.2176628746885205


# 4 bands OIF Calculation

In [95]:
combs_4 = findsubsets(band_numbers, 4)
oif_combs_4 = calculate_oif(combs_4, correlation_bands, std_bands, band_names)

In [104]:
oif_combs_4.sort()
print(oif_combs_4[0])
print(oif_combs_4[1])

[Band Models = B02_10, B02_20, B03_20, B04_20], OIF = 3.7310502542557495
[Band Models = B02_10, B02_20, B03_20, B05_20], OIF = 3.3156948620396354


# 6 bands OIF Calculation

In [96]:
combs_6 = findsubsets(band_numbers, 6)
oif_combs_6 = calculate_oif(combs_6, correlation_bands, std_bands, band_names)

In [105]:
oif_combs_6.sort()
print(oif_combs_6[0])
print(oif_combs_6[1])

[Band Models = B02_10, B02_20, B03_20, B04_20, B05_20, B06_20], OIF = 1.3592935947212559
[Band Models = B02_10, B02_20, B03_20, B04_20, B05_20, B07_20], OIF = 1.3143392960534808


# 9 bands OIF Calculation

In [98]:
combs_9 = findsubsets(band_numbers, 9)
oif_combs_9 = calculate_oif(combs_9, correlation_bands, std_bands, band_names)

In [106]:
oif_combs_9.sort()
print(oif_combs_9[0])
print(oif_combs_9[1])

[Band Models = B02_10, B02_20, B03_20, B04_20, B05_20, B06_20, B07_20, B11_20, B12_20], OIF = 0.5559136088191604
[Band Models = B02_10, B02_20, B03_20, B04_20, B05_20, B06_20, B07_20, B11_20, B8A_20], OIF = 0.5274140923030315
